#  Agents

## Functionalities:
- Task-based Agents which will break down tasks into subtasks and solve them in bite-sized portions
- Agents with registered functions as skills

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [1]:
# Set up API key and do the necessary imports
import os
from taskgen import *
os.environ['OPENAI_API_KEY'] = '<YOUR_API_KEY_HERE>'

# Agent Basics
- Create an agent by entering your agent's name and description
- Agents are task-based, so they will help generate subtasks to fulfil your main task

- Agents are made to be non-verbose, so they will just focus only on task instruction (Much more efficient compared to conversational-based agentic frameworks like AutoGen)
- Agent's interactions will be stored into `subtasks_completed` by default, which will serve as a memory buffer for future interactions

- **Inputs for Agent**:
    - **agent_name**: String. Name of agent, hinting at what the agent does
    - **agent_description**: String. Short description of what the agent does
    - **max_subtasks**: Int. Default: 5. The maximum number of subtasks the agent can have
    - **verbose**: Bool. Default: True. Whether to print out agent's intermediate thoughts
<br/><br/>

- **Agent Internal Parameters**:
    - **Task**: String. The task the agent has been assigned to - Defaults to "No task assigned"
    - **Subtasks Completed**: Dict. The keys are the subtask names and the values are the result of the respective subtask
    - **Is Task Completed**: Bool. Whether the current Task is completed
<br/><br/>

- **Automatic Task Running**
    - **reset()**: Resets the Agent Internal Parameters
    - **run(task: str)**: Performs the task. Do note that agent's state will not be reset, so if you want to reset it, call reset() prior to running this
<br/><br/>


- **Manual Task Running**
    - **assign(task: str)**: Assigns a task to the agent, setting Is Task Completed to False
    - **run(num_subtasks: int)**: Runs the task for num_subtasks number of subtasks. Uses the existing assigned task
<br/><br/>

- **Give User Output**
    - **reply_user(query: str = '', stateful: bool = True)**: Using all information from subtasks, give a reply about the `query` to the user. If `query` is not given, then it replies based on the current task the agent is doing. If `stateful` is True, saves this query and reply into `subtasks_completed`
<br/><br/>
    
- **Check status of Agent**:
    - **status()**: Lists out Agent Name, Agent Description, Available Functions (default function is to use the LLM), Task, Subtasks Completed and Is Task Completed
    
## Example Agent Creation
```python
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent')
```

## Example Agent Task Running - Split the assigned task into subtasks and execute each of them

```python
output = await my_agent.run('Give me 5 words rhyming with cool, and make a 4-sentence poem using them')
```

`Subtask identified: Find 5 words that rhyme with 'cool'`

`Getting LLM to perform the following task: Find 5 words that rhyme with 'cool'`
> pool, rule, fool, tool, school

`Subtask identified: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`Getting LLM to perform the following task: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`
> In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.

`Task completed successfully!`

## Example Agent Reply to User - Reference the subtasks' output to answer the user's query
```python
output = await my_agent.reply_user()
```

`
Here are 5 words that rhyme with "cool": pool, rule, fool, tool, school. Here is a 4-sentence poem using these words: "In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool."
`

## Check Agent's Status
```python
my_agent.status()
```

`Agent Name: Helpful assistant`

`Agent Description: You are a generalist agent`

`Available Functions: ['use_llm', 'end_task']`

`Task: Give me 5 words rhyming with cool, and make a 4-sentence poem using them`

`Subtasks Completed:`

`Subtask: Find 5 words that rhyme with 'cool'`

`pool, rule, fool, tool, school`

`Subtask: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.`

`Is Task Completed: True`

## Create Agent

In [3]:
# Create your agent by specifying name and description
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent')

In [4]:
# Show the agent status - By default agent comes equipped with default function `use_llm` which queries the llm
# end_task is to end the current task if it is completed
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


## Run Task! (Example 1)

In [6]:
# ASYNC
my_agent.reset()
output = await my_agent.run('Give me 5 words rhyming with cool, and then make a 4-sentence poem using them')
print(output)

Observation: No subtasks have been completed yet.
Thoughts: To complete the assigned task, I need to find 5 words rhyming with "cool" and then create a 4-sentence poem using them.
Subtask identified: Find 5 words rhyming with "cool".
Getting LLM to perform the following task: Find 5 words rhyming with "cool".
> The five words that rhyme with "cool" are: pool, drool, fool, school, and tool.

Observation: The five words that rhyme with "cool" have been found: pool, drool, fool, school, and tool.
Thoughts: To complete the assigned task, I need to create a 4-sentence poem using these words.
Subtask identified: Compose a 4-sentence poem using the words: pool, drool, fool, school, and tool.
Getting LLM to perform the following task: Compose a 4-sentence poem using the words: pool, drool, fool, school, and tool.
> In the cool pool, the fool from school used a tool to stir the water, making others drool.

Observation: The user requested 5 words rhyming with "cool" and a 4-sentence poem using t

In [7]:
output_async = await my_agent.reply_user()

In the cool pool, the fool from school used a tool to stir the water, making others drool.


In [11]:
# You can also ask your questions to the agent, and the agent will reply according to its current state. Functions like a QA bot
# Here, we set stateful is false because we do not want this to go into subtasks_completed
output = await my_agent.reply_user('Where is the pool?', stateful = False)

The pool is located in the backyard of a house, where people can swim and relax during hot days. It is a common feature in many residential properties and recreational facilities.


In [12]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Find 5 words rhyming with "cool".
{'Detailed Outcome': 'The five words that rhyme with "cool" are: pool, drool, fool, school, and tool.'}

Subtask: Compose a 4-sentence poem using the words: pool, drool, fool, school, and tool.
{'Detailed Outcome': 'In the cool pool, the fool from school used a tool to stir the water, making others drool.'}

Subtask: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
In the cool pool, the fool from school used a tool to stir the water, making others drool.

Is Task Completed: True


## Run Task! (Example 2)

In [16]:
# Create your agent by specifying name and description
my_agent = AsyncAgent('Trip Planner',
'''You are a planner for a trip.
When given a destination, first do an overall broad plan for all days with no mention of the time 
e.g. Day 1 - Going to City for Activities, Day 2 - Going to the Wild for Activities

Thereafter, fill in details for the morning, afternoon and evening activities, 
e.g. Day 1 Morning - Shopping, Afternoon - Eating, Evening - Theatre

Do not do anything other than generating the plan''')

In [17]:
# Do the task by subtasks
output = await my_agent.run('Give me a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing')

Observation: No subtasks have been completed yet.
Thoughts: I need to start by creating a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing.
Subtask identified: Generate a broad plan for a 3-day itinerary in Hawaii, involving scuba-diving and sightseeing.
Getting LLM to perform the following task: Generate a broad plan for a 3-day itinerary in Hawaii, involving scuba-diving and sightseeing.
> Day 1 - Explore the beautiful beaches of Hawaii, Day 2 - Enjoy scuba-diving in the crystal-clear waters, Day 3 - Visit the stunning waterfalls and lush forests. Day 1 Morning - Relaxing on the beach, Afternoon - Snorkeling in the coral reefs, Evening - Sunset cruise. Day 2 Morning - Scuba-diving adventure, Afternoon - Beach picnic, Evening - Luau dinner show. Day 3 Morning - Hiking to scenic viewpoints, Afternoon - Exploring local markets, Evening - Traditional Hawaiian dance performance.

Observation: The broad and detailed plan for a 3-d

In [20]:
# Generates a meaningful reply to the user
output =await my_agent.reply_user()

Day 1 - Explore the beautiful beaches of Hawaii, Day 2 - Enjoy scuba-diving in the crystal-clear waters, Day 3 - Visit the stunning waterfalls and lush forests. Day 1 Morning - Relaxing on the beach, Afternoon - Snorkeling in the coral reefs, Evening - Sunset cruise. Day 2 Morning - Scuba-diving adventure, Afternoon - Beach picnic, Evening - Luau dinner show. Day 3 Morning - Hiking to scenic viewpoints, Afternoon - Exploring local markets, Evening - Traditional Hawaiian dance performance.


In [21]:
my_agent.status()

Agent Name: Trip Planner
Agent Description: You are a planner for a trip.
When given a destination, first do an overall broad plan for all days with no mention of the time 
e.g. Day 1 - Going to City for Activities, Day 2 - Going to the Wild for Activities

Thereafter, fill in details for the morning, afternoon and evening activities, 
e.g. Day 1 Morning - Shopping, Afternoon - Eating, Evening - Theatre

Do not do anything other than generating the plan
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing
Subtasks Completed:
Subtask: Generate a broad plan for a 3-day itinerary in Hawaii, involving scuba-diving and sightseeing.
{'Detailed Outcome': 'Day 1 - Explore the beautiful beaches of Hawaii, Day 2 - Enjoy scuba-diving in the crystal-clear waters, Day 3 - Visit the stunning waterfalls and lush forests. Day 1 Morning - Relaxing on the beach

# Power Up your Agents - Bring in Functions (aka Tools)
- After creating your agent, use `assign_functions` to assign a list of functions (of class Function) to it
- Function names will be automatically inferred if not specified
- Proceed to run tasks by using `run()`

```python
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent')

my_agent.assign_functions([sentence_style, b2d])

output = await my_agent.run('Generate me a happy sentence with a number and a ball. The number is 1001 converted to decimal')
```

`Subtask identified: Convert the binary number 1001 to decimal`
`Calling function binary_to_decimal with parameters {'x': '1001'}`

> {'output1': 9}

`Subtask identified: Generate a happy sentence with the decimal number and a ball`
`Calling function sentence_with_objects_entities_emotion with parameters {'obj': '9', 'entity': 'ball', 'emotion': 'happy'}`

> {'output': 'I am so happy with my 9 balls.'}

`Task completed successfully!`

- Approach 1: Automatically Run your agent using `run()`

- Approach 2: Manually select and use functions for your task
    - **select_function(task: str)**: Based on the task, output the next function name and input parameters
    - **use_function(function_name: str, function_params: dict, subtask: str = '', stateful: bool = True)**: Uses the function named `function_name` with `function_params`. `stateful` controls whether the output of this function will be saved to `subtasks_completed` under the key of `subtask`
<br/><br/>

- **Assign/Remove Functions**:
    - **assign_functions(function_list: list)**: Assigns a list of functions to the agent
    - **remove_function(function_name: str)**: Removes function named function_name from the list of assigned functions
<br/><br/>

- **Show Functions**:
    - **list_functions()**: Returns the list of functions of the agent
    - **print_functions()**: Prints the list of functions of the agent
<br/><br/>

In [22]:
def binary_to_decimal(x):
    return int(str(x), 2)

sentence_style_async = AsyncFunction(fn_description = 'Output a sentence with <obj> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, fn_name = 'sentence_with_objects_entities_emotion')



b2d_async = AsyncFunction(fn_name = "binary_to_decimal",fn_description = 'Convert input <x: a binary number in base 2> to base 10', 
            output_format = {'output1': 'x in base 10'},
            external_fn = binary_to_decimal)

In [23]:
# Create your agent

my_agent= AsyncAgent(agent_name = 'Helpful assistant', 
      agent_description = 'You are a generalist agent')

In [24]:
# Assign functions
my_agent.assign_functions(function_list = [b2d_async, sentence_style_async])

In [25]:
# Show the functions the agent has
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: binary_to_decimal
Description: Convert input <x: a binary number in base 2> to base 10
Input: ['x']
Output: {'output1': 'x in base 10'}

Name: sentence_with_objects_entities_emotion
Description: Output a sentence with <obj> and <entity> in the style of <emotion>
Input: ['obj', 'entity', 'emotion']
Output: {'output': 'sentence'}



## Automatic Usage of Functions

In [26]:
# multi-task with multiple functions
output = await my_agent.run('Generate me a happy sentence with a number (convert from binary: 1001) and a ball')

Observation: No subtasks have been completed yet for the assigned task
Thoughts: To complete the assigned task, I need to generate a happy sentence with a number (convert from binary: 1001) and a ball. I can use the equipped function "binary_to_decimal" to convert the binary number to base 10 and then construct the sentence with the objects and emotion specified.
Subtask identified: Convert the binary number 1001 to base 10
Calling function binary_to_decimal with parameters {'x': 1001}
> {'output1': 9}

Observation: The binary number 1001 has been successfully converted to base 10, resulting in the number 9
Thoughts: To complete the assigned task, we need to generate a happy sentence with the number 9 (converted from binary: 1001) and a ball
Subtask identified: Output a sentence with "9" and "ball" in the style of "happy"
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 9, 'entity': 'ball', 'emotion': 'happy'}
> {'output': 'I am so happy to have 9 balls!'

In [27]:
# visualise the subtask outputs of the task
print(output)

[{'output1': 9}, {'output': 'I am so happy to have 9 balls!'}]


In [30]:
# give a response to user
output = await my_agent.reply_user()

I am so happy to have 9 balls!


# Manual Usage of Functions
- If you want to just see what the agent would choose for a hypothetical task, use `select_function`. This will not update the internal state, and will output function name and function params
- If you want specificity in the process, you can just execute the agent's functions yourself using `use_function`
- `use_function`: Uses an agent's function using name and params. Note that by default there will be updating of subtasks_completed when performing the function. In order not to update subtasks_completed, set `stateful = False`

In [32]:
# this should call generate_sentence_with_emotion
my_agent.reset()
function_name, function_params = await my_agent.select_function(
    task = 'Output a sentence with bell, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

await my_agent.use_function(function_name, function_params, stateful = False)

Observation: The assigned task is to output a sentence with bell, dog and happy.
Thoughts: To complete the assigned task, I need to generate a sentence that includes the objects "bell" and "dog" in a style that conveys "happy" emotion.
Selected function name: sentence_with_objects_entities_emotion
Selected function params: {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
> {'output': 'The dog wagged its tail happily when it heard the bell.'}



{'output': 'The dog wagged its tail happily when it heard the bell.'}

# Shared Variables

*"Because text is not enough"* - Anonymous

- `shared_variables` is a dictionary, that is initialised in Agent (default empty dictionary), and can be referenced by any function of the agent (including Inner Agents and their functions)
- This can be useful for non-text modalitiies (e.g. audio, pdfs, image) and lengthy text modalities, which we do not want to output into `subtasks_completed` directly
- `s_` at the start of the variable names means shared variables
    - For input, it means we take the variable from `shared_variables` instead of LLM generated input
    - For output, it means we store the variable into `shared_variables` instead of storing it in `subtasks_completed`. If `subtasks_completed` output is empty, it will be output as `{'Status': 'Completed'}`
- Example shared variables names: `s_sum`, `s_total`, `s_list_of_words`
- Note: For external functions, you need not put a `s_` at the start of the name of shared variables

### Example Input
```python
add = AsyncFunction(fn_description = "Add <increment: int> to <s_total>", 
              output_format = {"s_total": "Modified total"})

# Define the calculator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
my_agent = AsyncAgent('Calculator', 'Does computations', shared_variables = {'s_total': 0}).assign_functions([add])

output = await my_agent.run('Increment s_total by 1')

print('Shared Variables:', my_agent.shared_variables)
```

### Example Output
`Subtask identified: Add 1 to s_total`

`Calling function add_int_to_variable with parameters {'increment': 1}`
> {'Status': 'Completed'}

`Task completed successfully!`

`Shared Variables: {'s_total': 1}`


# Direct input and output link to shared variable
- Get shared variable as input from `shared_variables`, and output shared variable directly into `shared_variables`
- Both LLM-based functions and external functions can use this approach
- Used when you want to input and output to a shared pool of variables without letting the LLM know the details, e.g. store a persistent state throughout function calls

In [28]:
add = AsyncFunction(fn_description = "Add <increment: int> to <s_total>", 
              output_format = {"s_total": "Modified total"})
await add.async_init()

In [29]:
def multiply_count(multiplier: int, s_total: int):
    ''' Multiplies s_total by multiplier '''
    return s_total * multiplier

multiply = AsyncFunction(
    output_format = {"s_total": "Modified total"}, external_fn = multiply_count)

await multiply.async_init()


In [30]:
my_agent = AsyncAgent('Calculator', 'Does computations', default_to_llm = False, shared_variables = {'s_total': 0}).assign_functions([add, multiply])

In [31]:
print('Shared Variables Sync:', my_agent.shared_variables)


Shared Variables Sync: {'s_total': 0, 'agent': <taskgen.agent.AsyncAgent object at 0x11d628380>}


In [32]:
my_agent.function_map

{'end_task': <taskgen.function.AsyncFunction at 0x11d628590>,
 'add_increment_to_s_total': <taskgen.function.AsyncFunction at 0x11dee7950>,
 'multiply_count': <taskgen.function.AsyncFunction at 0x11de06f00>}

In [33]:
output = await my_agent.run('Increment s_total by 1')

Observation: No subtasks completed yet for Assigned Task
Thoughts: Need to increment s_total by 1
Subtask identified: Increment s_total by 1
Calling function add_increment_to_s_total with parameters {'increment': 1}
> {'Status': 'Completed'}

Observation: The subtask "Increment s_total by 1" has been completed
Thoughts: The next step is to end the task as there are no more subtasks remaining
Subtask identified: End Task
Task completed successfully!



In [34]:
my_agent.shared_variables

{'s_total': 1, 'agent': <taskgen.agent.AsyncAgent at 0x11d628380>}

In [35]:
my_agent.reset()
# When doing agents as OS, it is important to reset past subtasks completed so as not to influence future tasks
output = await my_agent.run('First increment s_total by 2, then multiply by 5')

Observation: No subtasks completed yet
Thoughts: Need to start by incrementing s_total by 2
Subtask identified: Add 2 to s_total
Calling function add_increment_to_s_total with parameters {'increment': 2}
> {'Status': 'Completed'}

Observation: The first subtask of incrementing s_total by 2 has been completed
Thoughts: The next step is to multiply the current s_total by 5 to complete the assigned task
Subtask identified: Multiply s_total by 5
Calling function multiply_count with parameters {'multiplier': 5}
> {'Status': 'Completed'}

Observation: Both subtasks (Add 2 to s_total and Multiply s_total by 5) have been completed for the Assigned Task
Thoughts: The Assigned Task is almost completed, only the final step of passing the final output to the user is left
Subtask identified: End Task
Task completed successfully!



In [36]:
my_agent.shared_variables

{'s_total': 15, 'agent': <taskgen.agent.AsyncAgent at 0x11d628380>}

#  Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.
    - Default: `memory_bank = {'Function': AsyncMemory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker')}`

## AsyncMemory Class
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 3. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `ranker`: `AsyncRanker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a async function which returns similarity score from 0 to 1 when given a query and key
        
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output =await my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```

`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`


In [59]:
# Define multiple functions
import math


fn_list = [
    AsyncFunction("Adds all numbers in <num_list: List[int]>", output_format = {"Result": "Sum of all numbers in num_list"}, 
        is_compulsory = True, external_fn = lambda num_list: sum(x for x in num_list), fn_name = 'sum_numbers'),
    AsyncFunction("Subtracts <num1: float> from <num2: float>", output_format = {"Result": "num1 - num2"}, fn_name = 'subtract_numbers', external_fn = lambda num1, num2: num1 + num2),
    AsyncFunction("Multiply <num1: float> by <num2: float>", output_format = {"Result": "num1 * num2"}, fn_name = 'multiply_numbers', external_fn = lambda num1, num2: num1 * num2),
    AsyncFunction("Divide <num1: float> by <num2: float>", output_format = {"Result": "num1 / num2"}, fn_name = 'divide_numbers', external_fn = lambda num1, num2: num1 / num2),
    AsyncFunction("Returns <num1: float>**<num2: float>", output_format = {"Result": "num1**num2"}, fn_name = 'power_operation', external_fn = lambda num1, num2: math.pow(num1,num2)),
    AsyncFunction("Returns Greatest Common Divisor of <num1: int> and <num2: int>", output_format = {"Result": "GCD(num1, num2)"}, fn_name = 'greatest_common_divisor', external_fn = lambda num1, num2: math.gcd(num1, num2)),
    AsyncFunction("Returns modulo of <num1: int> over <num2: int>", output_format = {"Result": "num1 % num2"}, fn_name = 'modulo_of_numbers', external_fn = lambda num1, num2: num1 % num2),
    AsyncFunction("Returns absolute difference between <num1: float> and <num2: float>", output_format = {"Result": "abs(num1 - num2)"}, fn_name = 'absolute_difference', external_fn = lambda num1, num2: math.abs(num1-num2)),
    AsyncFunction("Generates a poem containing <num1: float> and <num2: float>", output_format = {"Poem": "Poem"}, fn_name = 'generate_poem_with_numbers'),
    AsyncFunction("Lists out <num: int> words related to <word: str>", output_format = {"List of words": "List of words, type: list"}, fn_name = 'list_related_words'),
    AsyncFunction("Generates a quote about <topic: str>", output_format = {"Quote": "Quote"}, fn_name = 'generate_quote')
]

In [60]:
import asyncio



tasks = [f.async_init() for f in fn_list]
results = await asyncio.gather(*tasks)

In [61]:
my_agent = AsyncAgent('Generalist Agent', 
'''Does everything''',
                default_to_llm = False).assign_functions(fn_list)

In [62]:
my_agent.list_functions()

['Name: end_task\nDescription: Passes the final output to the user\nInput: []\nOutput: {}\n',
 "Name: sum_numbers\nDescription: Adds all numbers in <num_list: List[int]>\nInput: ['num_list']\nOutput: {'Result': 'Sum of all numbers in num_list'}\n",
 "Name: subtract_numbers\nDescription: Subtracts <num1: float> from <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 - num2'}\n",
 "Name: multiply_numbers\nDescription: Multiply <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 * num2'}\n",
 "Name: divide_numbers\nDescription: Divide <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 / num2'}\n",
 "Name: power_operation\nDescription: Returns <num1: float>**<num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1**num2'}\n",
 "Name: greatest_common_divisor\nDescription: Returns Greatest Common Divisor of <num1: int> and <num2: int>\nInput: ['num1', 'num2']\nOutput: {'Result': 'GCD(num1, num2)'}\n",
 "Name: 

In [63]:
[f.fn_name for f in await my_agent.memory_bank['Function'].retrieve_by_ranker('subtract numbers')]

['subtract_numbers',
 'divide_numbers',
 'multiply_numbers',
 'modulo_of_numbers',
 'generate_poem_with_numbers']

In [57]:
my_agent.reset()
await my_agent.run('Evaluate 2+3')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: No subtasks completed yet
Thoughts: Need to evaluate 2+3
Subtask identified: Adds all numbers in [2, 3]
Calling function sum_numbers with parameters {'num_list': [2, 3]}
> {'Result': 5}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: The sum of 2 and 3 has been calculated, resulting in 5
Thoughts: The task is to evaluate 2+3, which has been partially completed by adding the numbers together
Subtask identified: End Task
Task completed successfully!



[{'Result': 5}]

In [64]:
my_agent.reset()
output = await my_agent.run('Evaluate 2**10 * (3+5) / 10')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: No subtasks completed yet
Thoughts: Need to start evaluating the expression provided by the user
Subtask identified: Returns 2**10
Calling function power_operation with parameters {'num1': 2.0, 'num2': 10.0}
> {'Result': 1024.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: The subtask "Returns 2**10" has been completed successfully, resulting in 1024.0
Thoughts: To complete the remainder of the assigned task, we need to multiply the result of 1024.0 by (3+5) and then divide the final result by 10.
Subtask identified: Multiply 1024.0 by 8.0
Calling function multiply_numbers with parameters {'num1': 1024.0, 'num2': 8.0}
> {'Result': 8192.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, 

/var/folders/dh/kfw4jf5x6fx0t1khl3k97pq80000gn/T/ipykernel_80138/3032784745.py:2: RuntimeWarning: coroutine 'AsyncAgent.run' was never awaited
  output = await my_agent.run('Evaluate 2**10 * (3+5) / 10')


In [65]:
await my_agent.reply_user()

The result of evaluating 2**10 * (3+5) / 10 is 819.2


'The result of evaluating 2**10 * (3+5) / 10 is 819.2'